# Journal 2022-08-23

In [1]:
%load_ext autoreload
%autoreload 2

# Links
* [Symmetry and Topological phases (twitter thread)](https://twitter.com/MBarkeshli/status/1561417563619426309)
  * [2208.07367 Codimension-2 defects and higher symmetries in (3+1)D topological phases
](https://arxiv.org/abs/2208.07367)
* [Stable Diffusion - Explained (twitter thread)](https://twitter.com/ai__pub/status/1561362542487695360)
  * [2112.10752 High-Resolution Image Synthesis with Latent Diffusion Models](https://arxiv.org/abs/2112.10752) 
  

# DataCamp - Winning a Kaggle Competition


Capturing some of the code used to evaluate exercises as it's a nice template for evaulating performance of new features.  Will have to check SciKit Learn doc, this probably exists somewhere as a pipeline step for a model.

In [2]:
def get_kfold_rmse(train, features=None, target=None):
    #  Validation code from https://campus.datacamp.com/courses/winning-a-kaggle-competition-in-python/feature-engineering-6e5f9c7c-cf6c-4f79-b95c-fcf856dd17d7?ex=2
    #  Copied to have an example of simplified validation loop
    mse_scores = []

    for train_index, test_index in kf.split(train):
        train = train.fillna(0)
        feats = features or [x for x in train.columns if x not in ['Id', 'SalePrice', 'RoofStyle', 'CentralAir']]
        
        fold_train, fold_test = train.loc[train_index], train.loc[test_index]

        # Fit the data and make predictions
        # Create a Random Forest object
        rf = RandomForestRegressor(n_estimators=10, min_samples_split=10, random_state=123)

        # Train a model
        target = target or 'SalePrice'
        rf.fit(X=fold_train[feats], y=fold_train[target])

        # Get predictions for the test set
        pred = rf.predict(fold_test[feats])
    
        fold_score = mean_squared_error(fold_test[target], pred)
        mse_scores.append(np.sqrt(fold_score))
        
    return round(np.mean(mse_scores) + np.std(mse_scores), 2)

Target Encoding was something I learned from the course: for categorical variables you can use the training set group mean of each category as an encoding of the categorical feature.  Seems similar to an embedding?

In [ ]:
def test_mean_target_encoding(train, test, target, categorical, alpha=5):
    # From https://campus.datacamp.com/courses/winning-a-kaggle-competition-in-python/feature-engineering-6e5f9c7c-cf6c-4f79-b95c-fcf856dd17d7?ex=8
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
    
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

For the training encoding we need to use cross validation and take the mean value of the other folds

In [ ]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits=5, random_state=123, shuffle=True)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values